# Ferma Challenge — Как начать

Этот ноутбук поможет быстро стартовать с задачей по прогнозу годового удоя коровы. Здесь:

- краткое описание признаков;
- пример сохранения предсказаний для Kaggle (файл `ex.csv`)


## Описание данных

Каждая строка — отдельная корова.

| Column                 | Meaning                                             |
| ---------------------- | --------------------------------------------------- |
| `cow_id`               | Уникальный идентификатор коровы                     |
| `milk_yield_kg`        | Годовой удой (кг) — целевой признак                 |
| `feed_energy_eke`      | Энергетическая кормовая единица                     |
| `feed_crude_protein_g` | Содержание сырого протеина в корме (г)              |
| `sugar_protein_ratio`  | Сахаро-протеиновое соотношение                      |
| `breed`                | Порода коровы                                       |
| `pasture_type`         | Тип пастбища                                        |
| `sire_breed`           | Порода отца                                         |
| `milk_fat_pct`         | Жирность молока (%)                                 |
| `milk_protein_pct`     | Белок в молоке (%)                                  |
| `milk_taste_label`     | Качественная оценка вкуса                           |
| `age_group`            | Возрастная группа (`under_2_years`, `over_2_years`) |

Цель: построить модель, минимизирующую **MAE** при прогнозировании `milk_yield_kg`.


## МОЖНО НАЧИНАТЬ


In [7]:
import pandas as pd

train_path = "../data/train.csv"
test_path = "../data/test.csv"

df = pd.read_csv(train_path, sep=",", decimal=".")
test_df = pd.read_csv(test_path, sep=",", decimal=".")

print(df.shape, test_df.shape)

df.head()

(507, 12) (126, 11)


,cow_id,milk_yield_kg,feed_energy_eke,feed_crude_protein_g,sugar_protein_ratio,breed,pasture_type,sire_breed,milk_fat_pct,milk_protein_pct,milk_taste_label,age_group
0,488,6005,13.5,1842,0.940,РефлешнСоверинг,Равнинное,Айдиал,3.62,3.073,не вкусно,более_2_лет
1,422,5982,13.8,1722,0.890,РефлешнСоверинг,Холмистое,Айдиал,3.61,3.073,не вкусно,более_2_лет
2,105,5700,14.4,1934,0.885,Вис Бик Айдиал,Равнинное,Соверин,3.57,3.079,вкусно,более_2_лет
3,115,5412,12.1,1924,0.890,Вис Бик Айдиал,Равнинное,Соверин,3.57,3.071,не вкусно,менее_2_лет
4,350,6171,15.3,1966,0.940,РефлешнСоверинг,Равнинное,Соверин,3.73,3.076,вкусно,более_2_лет


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cow_id                507 non-null    int64  
 1   milk_yield_kg         507 non-null    int64  
 2   feed_energy_eke       507 non-null    float64
 3   feed_crude_protein_g  507 non-null    int64  
 4   sugar_protein_ratio   507 non-null    float64
 5   breed                 507 non-null    object 
 6   pasture_type          507 non-null    object 
 7   sire_breed            507 non-null    object 
 8   milk_fat_pct          507 non-null    float64
 9   milk_protein_pct      507 non-null    float64
 10  milk_taste_label      507 non-null    object 
 11  age_group             507 non-null    object 
dtypes: float64(4), int64(3), object(5)
memory usage: 47.7+ KB


In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

target_col = "milk_yield_kg"

feature_cols = [c for c in df.columns if c not in ["cow_id", target_col]]

X = df[feature_cols]
y = df[target_col]

# числ/кат
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in feature_cols if c not in num_cols]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop",
)

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

candidates = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
}

best_name = None
best_pipe = None
best_mae = float("inf")

for name, model in candidates.items():
    pipe = Pipeline(
        steps=[
            ("preprocess", preprocess),
            ("model", model),
        ]
    )

    pipe.fit(X_train, y_train)
    val_pred = pipe.predict(X_val)

    cur_mae = mean_absolute_error(y_val, val_pred)

    if cur_mae < best_mae:
        best_mae = cur_mae
        best_name = name
        best_pipe = pipe

val_pred = best_pipe.predict(X_val)
mae = mean_absolute_error(y_val, val_pred)
rmse = np.sqrt(mean_squared_error(y_val, val_pred))
r2 = r2_score(y_val, val_pred)

print(f"MAE : {mae}")
print(f"RMSE: {rmse}")
print(f"R^2 : {r2}")

best_pipe.fit(X, y)
full_predictions = best_pipe.predict(test_df[feature_cols])

# сегодня без отрицательного молока(
full_predictions = np.clip(full_predictions, a_min=0, a_max=None)

Best model: Ridge
MAE : 475.97686219457603
RMSE: 645.8216760765067
R^2 : -0.5657113195125736


### Обязательные метрики

```
print(f"MAE : {mae}")
print(f"RMSE: {rmse}")
print(f"R^2 : {r2}")
```


## Подготовка файла для Kaggle

Сохраняем файл с колонками `cow_id` и `milk_yield_prediction`.


In [11]:
submission = pd.DataFrame(
    {
        "cow_id": test_df["cow_id"],
        "milk_yield_prediction": full_predictions,
    }
)

submission.head()

,cow_id,milk_yield_prediction
0,397,6295.868129
1,249,5961.481747
2,216,6027.590387
3,354,6326.933869
4,549,6638.229205


In [12]:
submission_path = "../submissions/ex.csv"
submission.to_csv(submission_path, index=False)
print(f"Saved baseline submission to {submission_path}")

Saved baseline submission to ../submissions/ex.csv
